In [3]:
import os
import json
import geojson

In [4]:
pth_WSI_segmentations = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg'

In [5]:
json_pth_list = sorted([os.path.join(pth_WSI_segmentations,file) for file in os.listdir(pth_WSI_segmentations) if file.endswith(".json")])
print(json_pth_list)

['\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_14_1F_cropped.json', '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_23_4O_cropped.json', '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_29_3M_cropped.json', '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_30_3D_cropped.json', '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_34_3L_cropped.json', '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_36_5O_cropped.json', '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl

In [29]:
def get_ds_data(segmentation_data, ds):
    data_list = []
    for data in segmentation_data:
        centroid = data['centroid']
        contour = data['contour']
    
        #print(centroid)
        #print(contour)
        
        ds_centroid = [int(c/ds) for c in centroid]
        ds_contour = [[value / ds for value in sublist] for sublist in contour]
        ds_contour = [[round(x,2), round(y,2)] for x, y in ds_contour]
        #ds_contour = ds_contour[0:-1:4]  # make shape have 8 points instead of 32
        
        #print(ds_centroid)
        #print(ds_contour)
        
        dat = [ds_centroid, ds_contour]
        data_list.append(dat)
    return data_list

In [34]:
out_pth = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\qupath_geojsons'

ds_amt = 1/0.4416 # 10x

# for p, file in enumerate(json_pth_list):
for p, file in enumerate(json_pth_list):
    nm = file.split('\\')[-1]
    new_fn = os.path.join(out_pth, nm[:-5] + '.geojson')
    print(f'{p} / {len(json_pth_list)}')
    print(nm)
    
    if not os.path.exists(new_fn):
    
        segmentation_data = json.load(open(file))
        
        data_list = get_ds_data(segmentation_data, ds_amt)
    
        GEOdata = []
        
        for j, (centroid, contour) in enumerate(data_list):
            
            #if j == 100000:
            #    break
            
            centroid = [centroid[0] + 0, centroid[1] + 0]
            # xy coordinates are swapped, so I reverse them here with xy[::-1]
            # note: add 1 to coords to fix 0 indexing vs 1 index offset
            contour = [[coord for coord in xy[::-1]] for xy in contour]  # Convert coordinates to integers
            contour.append(contour[0]) # stardist doesn't close the circle, needed for qupath
        
            # Create a new dictionary for each contour
            dict_data = {
                "type": "Feature",
                "id": "PathCellObject",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [contour]
                },
                "properties": {
                    'objectType': 'annotation',
                    'classification': {'name': 'Nuclei', 'color': [97, 214, 59]}
                }
            }
        
            GEOdata.append(dict_data)
        
        with open(new_fn,'w') as outfile:
            geojson.dump(GEOdata,outfile)
        print('Finished',new_fn)
    
    else:
        print(f'skipping {new_fn}')
    #break

0 / 10
PD_14_1F_cropped.json
skipping \\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\qupath_geojsons\PD_14_1F_cropped.geojson
1 / 10
PD_23_4O_cropped.json
Finished \\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\qupath_geojsons\PD_23_4O_cropped.geojson
2 / 10
PD_29_3M_cropped.json
Finished \\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\qupath_geojsons\PD_29_3M_cropped.geojson
3 / 10
PD_30_3D_cropped.json
Finished \\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\qupath_geojsons\PD_30_3D_cropped.geojson
4 / 10
PD_34_3L_cropped.json
Finished \\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\qupath_geojsons\PD_34_3L_cropped.geojson
5 / 10
PD_36_5O_cropped.json
Finished \\10.162.80.

In [32]:
data_list

[[[187, 31],
  [[187.29, 33.23],
   [186.87, 33.09],
   [186.4, 33.0],
   [186.2, 32.54],
   [186.02, 32.15],
   [186.21, 31.64],
   [186.03, 31.35],
   [186.01, 31.01],
   [186.0, 30.66],
   [186.15, 30.35],
   [186.0, 29.85],
   [186.3, 29.57],
   [186.54, 29.22],
   [187.04, 29.31],
   [187.36, 29.12],
   [187.74, 28.97],
   [188.12, 29.07],
   [188.46, 29.27],
   [188.97, 29.16],
   [189.24, 29.51],
   [189.71, 29.68],
   [189.73, 30.18],
   [190.03, 30.55],
   [190.05, 31.01],
   [189.94, 31.44],
   [189.64, 31.79],
   [189.6, 32.25],
   [189.19, 32.46],
   [188.96, 32.83],
   [188.51, 32.87],
   [188.16, 33.15],
   [187.74, 33.25]]],
 [[112, 225],
  [[112.21, 227.59],
   [111.78, 227.45],
   [111.23, 227.46],
   [110.8, 227.17],
   [110.32, 226.88],
   [110.08, 226.38],
   [109.96, 225.85],
   [109.95, 225.31],
   [110.17, 224.81],
   [110.52, 224.42],
   [110.68, 223.98],
   [111.17, 223.82],
   [111.44, 223.48],
   [111.98, 223.66],
   [112.3, 223.5],
   [112.66, 223.47],
   [1

In [33]:
with open(new_fn,'w') as outfile:
    geojson.dump(GEOdata,outfile)
print('Finished',new_fn)

Finished \\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\qupath_geojsons\PD_14_1F_cropped.geojson
